# Deep Convolutional GAN 
Reference : https://arxiv.org/abs/1511.06434

### Target 
DC GAN on celebA, cifar-10, cifar-100 datasets
###### To report
+ Train - parameters and architecture 
+ saved weights for 3 datasets 
+ loss, number of epochs
+ generated images : modes vs quality

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
import tensorflow as tf
import numpy as np

In [9]:
(x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train/255, x_test/255

In [11]:
print(x_train.shape, x_test.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3)


#### plan

+ read to tf.Dataset() format
+ model class()
+ `noise(batch_size, dim, distribution='uniform'):`  
+ def `generator(z):` returns image
+ def `discriminator(img):` returns logits
+ def `batch_norm` : returns normalized batch
+ def `**LOSS**(real, fake):` returns losses of G and D
+ def `save():` saves model to a file 
+ def `load(filename):` loads saved model
+ def `generate_images(num_imgs)` - function to generate from trained model -
+ def `evaluate(x_test)`

#### Architecture guidelines for stable Deep Convolutional GANs - reference: GAN paper
+ Replace any pooling layers with strided convolutions (discriminator) and fractional-strided convolutions (generator).
+ Use batchnorm in both the generator and the discriminator. _Do not apply batchnorm to G output layer and D input layer_
+ Remove fully connected hidden layers for deeper architectures.
+ Use ReLU activation in generator for all layers except for the output, which uses Tanh.
+ Use LeakyReLU activation in the discriminator for all layers. In the LeakyReLU, the slope of the leak was set to 0.2 in all models.
+ Scale images to range of $[-1,1]$
+ minibatch stocastic gradient - batch_size = 128
+ All weights were initialized from a zero-centered Normal distribution with standard deviation 0.02.

+ Adam optimizer: learning_rate = 0.0002, β1 = 0.5
+ We found global average pooling increased model stability but hurt convergence speed. A middle ground of directly connecting the highest convolutional features to the input and output respectively of the generator and discriminator worked well.
+ simple image de-duplication process. We fit a 3072-128-3072 de-noising dropout regularized RELU autoencoder on 32x32 downsampled center-crops of training examples.
+ 

In [15]:
class DC_GAN():
    def __init__(self, sess, real_img_dim, gen_img_dim, noise_dim, batch_size, num_channels=3):
        self.sess = sess
        self.noise_dim = noise_dim
        self.D_input_dim = real_img_dim
        self.G_output_dim = gen_img_dim
        self.batch_size = batch_size
        self.channels = num_channels
        self.saver = tf.train.Saver()
        
    def noise(self, pdf='uniform'): 
        '''
        Inputs: batch size while training, dimension of noise
        returns random sample from the distribution - pdf (uniform by default)
        '''
        if pdf=='uniform':
            return tf.random_uniform(minval=-1, maxval=1, shape=(self.batch_size, self.noise_dim), name='z')
        if pdf=='normal':
            return tf.random_normal(mean=0, stddev=1, shape=(self.batch_size, self.noise_dim), name='z')
    
    def batch_norm(self, batch):
        return tf.contrib.layers.batch_norm(inputs=batch,
            decay=0.9,
            updates_collections=None,
            epsilon=1e-5,
            scale=True,
            is_training=True
            )
    
    def add_conv_layer(self, inputs, filter_size, in_channels, out_channels, scope_name, _batch_norm=True):
        with tf.variable_scope(scope_name):
            shape = [filter_size, filter_size, in_channels, out_channels]
            weights = tf.Variable(tf.truncated_normal(shape, stddev=0.02), trainable=True)
            biases = tf.Variable(tf.constant(0.05, shape=[out_channels]), trainable=True)
            
            layer = tf.nn.conv2d(
                input=inputs,
                filter=weights,
                strides=[1, 2, 2, 1],     
          errD_fake = self.d_loss_fake.eval({
              self.z: batch_z, 
              self.y:batch_labels
          })
                padding='SAME') + biases
            if _batch_norm: layer = self.batch_norm(layer)
            layer = tf.nn.leaky_relu(layer, alpha=0.2)
            return layer, weights, biases
            
    def add_conv_transpose_layer(self,inputs,filter_size,in_channels,out_channels,scope_name,activation,_batch_norm=True):
        with tf.variable_scope(scope_name):
            shape = [filter_size, filter_size, out_channels, in_channels]
            weights = tf.Variable(tf.truncated_normal(shape, stddev=0.02), trainable=True)
            biases = tf.Variable(tf.constant(0.05, shape=[out_channels]), trainable=True)

            layer = tf.nn.conv2d_transpose(
                input=inputs,
                filter=weights,
                strides=[1, 2, 2, 1],
                padding='SAME') + biases
            if _batch_norm: layer = self.batch_norm(layer)
            if activation == 'relu': layer = tf.nn.relu(layer)
            elif activation == 'tanh': layer = tf.nn.tanh(layer)

            return layer, weights, biases

    def discriminator(self, img):
        scope_name='discriminator'
        h1, w1, b1 = self.add_conv_layer(
            inputs = img,
            filter_size=5,
            in_channels=self.num_channels,
            out_channels=64,
            _batch_norm=False,
            scope_name=scope_name
        )
        h2, w2, b2 = self.add_conv_layer(
            inputs=h1,
            filter_size=5,
            in_channels=64,
            num_filters=128,
            scope_name=scope_name
        )
        h3, w3, b3 = self.add_conv_layer(
            inputs=h2,
            filter_size=5,
            in_channels=128,
            out_channels=256,
            scope_name=scope_name,
        )
        logits = tf.layers.dense(
            inputs=tf.reshape(h3, shape=[self.batch_size, -1]),
            units=1,
            use_bias=True,
            kernel_initalizer=tf.contrib.layers.xavier_initializer()
        )
        return logits
        
    def generator(self, z):
        scope_name = 'generator'
        _z = tf.layers.dense(
            inputs=z,
            units=4*4*1024,
            use_bias=True,
            kernel_initalizer=tf.contrib.layers.xavier_initializer()
        ) # projection of z to 4x4x1024 layer (linear)
        h1 = tf.reshape(_z, [-1, 4,4,1024])
        h2, w2, b2 = self.add_conv_transpose_layer(
            inputs=h1,
            filter_size=5,
            in_channels=1024,
            out_channels=512,
            scope_name=scope_name,
            activation='relu'
        )
        h3, w3, b3 = self.add_conv_transpose_layer(
            inputs=h2,
            filter_size=5,
            in_channels=512,
            out_channels=256,
            scope_name=scope_name,
            activation='relu'
        )
        G_out, w4, b4 = self.add_conv_transpose_layer(
            inputs=h3,
            filter_size=5,
            in_channels=256,
            num_filters=self.num_channels,
            _batch_norm=False,
            scope_name=scope_name,
            activation='tanh'
        )
        return G_out
    
    def generate_imgs(self, num_imgs):
        z = self.noise(num_imgs, self.noise_dim)
        gen_imgs = self.generator(z)
        return gen_imgs
        
    def compile_model(self, real_imgs, z):
        # tf.reset_default_graph()
        # self.real_imgs = tf.placeholder(dtype=tf.float32,shape=[self.batch_size]+ self.real_img_dim,name='real')
        # self.z = tf.placeholder(dtype=tf.float32, shape=[self.batch_size, self.noise_dim], name='noise')
        self.fake_imgs = self.generator(z)
        self.logits_real = self.discriminator(real_imgs)
        self.logits_fake = self.discriminator(self.fake_imgs)
        self.D_loss_real = tf.nn.sigmoid_cross_entropy_with_logits(
            logits=self.logits_real, labels=tf.ones_like(self.logits_real))
        self.D_loss_fake = tf.nn.sigmoid_cross_entropy_with_logits(
            logits=self.logits_fake, labels=tf.zeros_like(self.logits_fake))
        self.D_loss = tf.reduce_mean(self.D_loss_real) + tf.reduce_mean(self.D_loss_fake)
        self.G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=self.logits_fake, labels=tf.ones_like(self.logits_fake)))
        self.D_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'discriminator')
        self.G_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'generator')
    
    def preprocessing(self, x):
        return 2*x - 1;
    
    def train(self, real_data, learning_rate=0.0002, beta1=0.5, epochs):
        # create tf.data.Dataset() instance
        train_data = tf.data.Dataset.from_tensor_slices(real_data) 
        train_data = train_data.map(self.preprocessing, num_parallel_calls=self.batch_size)
        
        def noise_generator():
            _z = self.noise()
            yield _z
        z_sample = tf.data.Dataset.from_generator(generator=noise_generator, output_types=(tf.float32)) 
        # D train step
        self.D_train = tf.train.AdamOptimizer(
            learning_rate=learning_rate,
            beta1=beta1).minimize(self.D_loss, var_list=self.D_vars)
        # G train step
        self.G_train = tf.train.AdamOptimizer(
            learning_rate=learning_rate,
            beta1=beta1).minimize(self.G_loss, var_list=self.G_vars)
        
        tf.global_variables_initializer().run()
        train_data = train_data.shuffle(
            buffer_size=self.batch_size).repeat(count=epochs).batch(batch_size=self.batch_size)
        
        
        for epoch in range(epochs):
            

SyntaxError: unexpected EOF while parsing (<ipython-input-15-434c16df7337>, line 200)